# pytorch入門
簡単なmnistサンプルで学ぶ。あえて、convolutionやpoolingは使わない。
- 参考：[公式のcifar10サンプル](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html)

In [1]:
from tqdm.notebook import tqdm

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

## データ準備

In [3]:
# データの変換用
transform = transforms.Compose(
    [
        # numpy.ndarrayをテンソルに変換
        transforms.ToTensor(),
        # 平均0.5、標準偏差0.5へ。チャンネル数のタプルで渡す。
        transforms.Normalize(mean=(0.5, ), std=(0.5, ))
    ]
)
# 学習データ用意
trainset = torchvision.datasets.MNIST(
    root='./dataset', # 保存先
    train=True,
    download=True,
    transform=transform
)
# 学習用ローダー
trainloader = torch.utils.data.DataLoader(
    trainset,
    batch_size=128,
    shuffle=True,
    num_workers=2
)
# テストデータ用意
testset = torchvision.datasets.MNIST(
    root='./dataset', 
    train=False, 
    download=True, 
    transform=transform
)
# テスト用ローダー
testloader = torch.utils.data.DataLoader(
    testset, 
    batch_size=128,
    shuffle=False, 
    num_workers=2
)

### データの確認

In [4]:
dataiter = iter(trainloader)
images, labels = dataiter.next()

In [5]:
# バッチサイズ分
labels

tensor([2, 8, 3, 3, 3, 7, 6, 9, 6, 6, 9, 6, 4, 3, 0, 7, 8, 8, 5, 7, 6, 0, 2, 6,
        0, 6, 7, 5, 6, 9, 6, 4, 5, 8, 5, 2, 5, 7, 9, 4, 5, 5, 8, 5, 7, 7, 6, 9,
        9, 2, 6, 5, 3, 9, 8, 0, 4, 9, 8, 0, 1, 8, 7, 8, 6, 6, 3, 3, 9, 1, 0, 2,
        1, 9, 9, 9, 1, 8, 2, 3, 6, 6, 2, 1, 1, 8, 3, 4, 3, 9, 3, 7, 9, 1, 8, 1,
        5, 7, 8, 8, 5, 8, 1, 3, 2, 2, 4, 8, 9, 3, 1, 8, 9, 0, 8, 6, 4, 3, 7, 9,
        4, 4, 6, 6, 8, 8, 9, 6])

In [6]:
# バッチサイズ x チャンネル数 x height x width
images.shape

torch.Size([128, 1, 28, 28])

## モデル

In [7]:
import torch.nn as nn
import torch.nn.functional as F

# ネットワークの定義
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 28x28次元 -> 256次元
        self.fc1 = nn.Linear(1*28*28, 256)
        # 256次元 -> 10次元（10クラスあるため）
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        # shapeを変形する（バッチサイズ x 次元数）
        x = x.view(-1, 1*28*28)
        # 線形変換後ReLuをかます
        x = F.relu(self.fc1(x))
        # 【注意】Kerasだと最後にsoftmaxをかますが、
        # pytorchだと損失関数を計算する nn.CrossEntropyLoss の中にsoftmaxの計算が含まれている
        # ただし、予測時に確率にしたい場合はアウトプットを nn.functional.softmax() で変換する必要がある
        x = self.fc2(x)
        return x

In [8]:
net = Net()

In [9]:
import torch.optim as optim

# 目的関数
criterion = nn.CrossEntropyLoss()
# 最適化手法
optimizer = optim.Adam(net.parameters(), lr=0.001)

## 学習

In [10]:
# Kerasやsklearnでいうfit
epochs=10
varbose=100
for epoch in tqdm(range(epochs)):
    # 損失（表示用）
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        # 勾配の初期化
        optimizer.zero_grad()

        # Netのforwardに基づいて出力が計算される
        outputs = net(inputs)
        # 損失の計算
        loss = criterion(outputs, labels)
        # ↑ここまでで計算グラフの構築が行われ、勾配に使う情報が設定される

        # 勾配を計算
        loss.backward()
        # 最適化法に基づいてパラメータ更新
        optimizer.step()

        # varbose回分の損失の和。item() で値を取り出している
        running_loss += loss.item()
        if i % varbose == varbose-1:
            print(f"epoch:{epoch + 1}, batch:{i + 1}, loss: {running_loss / varbose}")
            running_loss = 0.0

epoch:1, batch:100, loss: 0.6820216017961502
epoch:1, batch:200, loss: 0.3693145060539246
epoch:1, batch:300, loss: 0.332705043554306
epoch:1, batch:400, loss: 0.29559941463172434
epoch:2, batch:100, loss: 0.23795114882290364
epoch:2, batch:200, loss: 0.20222241077572106
epoch:2, batch:300, loss: 0.1960952030867338
epoch:2, batch:400, loss: 0.1974168461561203
epoch:3, batch:100, loss: 0.15257078558206558
epoch:3, batch:200, loss: 0.1440433470159769
epoch:3, batch:300, loss: 0.142299246750772
epoch:3, batch:400, loss: 0.13810560397803784
epoch:4, batch:100, loss: 0.11472118373960256
epoch:4, batch:200, loss: 0.10906709022819996
epoch:4, batch:300, loss: 0.11881133031100034
epoch:4, batch:400, loss: 0.10873796993866564
epoch:5, batch:100, loss: 0.09303514676168562
epoch:5, batch:200, loss: 0.09735027667135
epoch:5, batch:300, loss: 0.09419337672181427
epoch:5, batch:400, loss: 0.08241723172366619
epoch:6, batch:100, loss: 0.07430373188108205
epoch:6, batch:200, loss: 0.07744961531832814


## 予測

In [11]:
correct = 0
total = 0
# 予測時に勾配更新しないため、no_gradを指定
with torch.no_grad():
    for (images, labels) in testloader:
        # forwardした結果
        # これを nn.functional.softmax() すると確率が得られるが、ここでは使わない
        outputs = net(images)
        # torch.max() は (最大値, そのindex)を返す関数。dimは最大値を見る軸
        _, predicted = torch.max(outputs.data, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f"Accuracy: {100 * float(correct/total)}")

Accuracy: 97.26


## GPUを使うには
コードの書き方だけメモ。環境構築は別のサイトを参照。

In [12]:
# cudaが使えるか確認
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [13]:
# .toメソッドでデバイスに移すようにする
# ネットワークに関して
net = Net().to(device)
# データに関して
images, labels = dataiter.next()
images, labels = images.to(device), labels.to(device)